<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Pytorch_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --upgrade fastai

     |████████████████████████████████| 194kB 5.4MB/s 
     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 12.8MB 234kB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [1]:
from fastai import *
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.IMAGENETTE_160)
path.ls()

In [18]:
path.ls()

(#4) [Path('/root/.fastai/data/imagenette2-160/noisy_imagenette.csv'),Path('/root/.fastai/data/imagenette2-160/.DS_Store'),Path('/root/.fastai/data/imagenette2-160/train'),Path('/root/.fastai/data/imagenette2-160/val')]

In [17]:
(path/'train'/'n03417042').ls()

(#961) [Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_12168.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_12235.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_17037.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_3107.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_23445.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_15484.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_4238.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_10796.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_21705.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03417042/n03417042_28584.JPEG')...]

In [24]:
train_path = path/'train'
[img for img in train_path.ls()]

[Path('/root/.fastai/data/imagenette2-160/train/n03417042'),
 Path('/root/.fastai/data/imagenette2-160/train/n03425413'),
 Path('/root/.fastai/data/imagenette2-160/train/.DS_Store'),
 Path('/root/.fastai/data/imagenette2-160/train/n02979186'),
 Path('/root/.fastai/data/imagenette2-160/train/n02102040'),
 Path('/root/.fastai/data/imagenette2-160/train/n01440764'),
 Path('/root/.fastai/data/imagenette2-160/train/n03394916'),
 Path('/root/.fastai/data/imagenette2-160/train/n03028079'),
 Path('/root/.fastai/data/imagenette2-160/train/n03888257'),
 Path('/root/.fastai/data/imagenette2-160/train/n03000684'),
 Path('/root/.fastai/data/imagenette2-160/train/n03445777')]

In [26]:
import re
re.sub('.*/', '', '/root/.fastai/data/imagenette2-160/train/n03417042')

'n03417042'

In [43]:
def get_folder_data(path):
    label = re.sub('.*/', '', str(path))
    return path.ls(), [label]*len(path.ls())

In [ ]:
list(get_folder_data((path/'train/n03445777')))

In [37]:
label = re.sub('.*/', '', str((path/'train/n03445777')))
label

'n03445777'

In [38]:
len((path/'train/n03445777').ls())

951

In [39]:
label*911

'n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777n03445777

In [ ]:
class myDatasetClass(img_path):
    def __init__(self, img_path, transform=None, target_transform=None):
        self.images = 
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return

In [31]:
(path/'train/n03445777').ls()

(#951) [Path('/root/.fastai/data/imagenette2-160/train/n03445777/ILSVRC2012_val_00031486.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_9118.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_10039.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_12117.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_743.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_5665.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_6038.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_3927.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_4475.JPEG'),Path('/root/.fastai/data/imagenette2-160/train/n03445777/n03445777_15388.JPEG')...]